# Pandas SQLite3

In [1]:
import pandas as pd
import numpy as np
import sqlite3

### Example 1

In [2]:
columns = [ "rowid","empid","name","managerid","dateOfjoin","city"]

rowid = [0,1,2,3]
empid =  [121,221,321,421]
name  = ['nurur rahman', 'abdur rahman', 'britany spears', 'sam adams']
managerid = [321,986,876,986]
city      = ['Toronto', 'New York','Los Angeles', 'New York']
dateOfjoin= ['01/31/2014','02/30/2015','11/27/2016','12/30/2018']


values = [rowid,empid,name,managerid,dateOfjoin,city]

df1  =  pd.DataFrame({columns[i]: values[i] for i in range(6)}, columns=columns)
df1

,rowid,empid,name,managerid,dateOfjoin,city
0,0,121,nurur rahman,321,01/31/2014,Toronto
1,1,221,abdur rahman,986,02/30/2015,New York
2,2,321,britany spears,876,11/27/2016,Los Angeles
3,3,421,sam adams,986,12/30/2018,New York


In [3]:
columns  = [ "empid","project","grade","salary","variable"]

empid    = [121,221,321,571,413,460,390]
project  = ['p1','p2','p3','p2','p3','p1','p2']
grade    = ['a','b','a','c','c','c','b']
salary   = [8000, 10000, 12000, 11500, 12000,11500,10100]
variable = [500, 800,90,100,120,150,140]

values = [empid,project,grade,salary,variable]

df2  =  pd.DataFrame({columns[i]: values[i] for i in range(5)}, columns=columns)
df2

,empid,project,grade,salary,variable
0,121,p1,a,8000,500
1,221,p2,b,10000,800
2,321,p3,a,12000,90
3,571,p2,c,11500,100
4,413,p3,c,12000,120
5,460,p1,c,11500,150
6,390,p2,b,10100,140


In [4]:
conn = sqlite3.connect(':memory:')

df1.to_sql('employeeTable', conn)
df2.to_sql('salaryTable', conn)

7

First and Second highest salaries

In [5]:
myquery="""
SELECT * FROM
salaryTable 
WHERE salary in
(SELECT 
DISTINCT(salary) unique_salary 
FROM salaryTable
ORDER BY unique_salary DESC 
LIMIT 2)  
ORDER BY salary DESC 
""" 

pd.read_sql(myquery, conn)

,index,empid,project,grade,salary,variable
0,2,321,p3,a,12000,90
1,4,413,p3,c,12000,120
2,3,571,p2,c,11500,100
3,5,460,p1,c,11500,150


Only the Second highest salary

In [6]:
myquery="""
SELECT * FROM
salaryTable 
WHERE salary in
(SELECT 
DISTINCT(salary) unique_salary 
FROM salaryTable
ORDER BY unique_salary DESC 
LIMIT 1 OFFSET 1)
ORDER BY salary DESC
""" 
pd.read_sql(myquery, conn)

,index,empid,project,grade,salary,variable
0,3,571,p2,c,11500,100
1,5,460,p1,c,11500,150


In [7]:
myquery="""

SELECT 
*
FROM
(SELECT 
salary as unique_salary,
COUNT(*) as count_unique_salary
FROM salaryTable
GROUP BY salary 
ORDER BY salary DESC) as f1
WHERE f1.count_unique_salary==1
""" 

pd.read_sql(myquery, conn)

,unique_salary,count_unique_salary
0,10100,1
1,10000,1
2,8000,1


####  Window Functions
Average salary by grade

In [8]:
myquery="""
SELECT 
grade, 
salary,
AVG(salary) OVER (PARTITION BY grade ORDER BY salary) as avg_salary
FROM
salaryTable 
""" 
pd.read_sql(myquery, conn)

,grade,salary,avg_salary
0,a,8000,8000.000000
1,a,12000,10000.000000
2,b,10000,10000.000000
3,b,10100,10050.000000
4,c,11500,11500.000000
5,c,11500,11500.000000
6,c,12000,11666.666667


In [9]:
myquery="""
SELECT 
grade, 
AVG(salary) OVER win     as avg_salary,
ROW_NUMBER() OVER win    as row_num,
RANK() OVER win          as rank_val,
DENSE_RANK() OVER win    as dense_rank_val,
PERCENT_RANK() OVER win  as  percent_rank_val,
CUME_DIST() OVER win     as cume_dist_val
FROM
salaryTable 
WINDOW win as (PARTITION BY grade ORDER BY salary)
""" 
pd.read_sql(myquery, conn)

,grade,avg_salary,row_num,rank_val,dense_rank_val,percent_rank_val,cume_dist_val
0,a,8000.000000,1,1,1,0.0,0.500000
1,a,10000.000000,2,2,2,1.0,1.000000
2,b,10000.000000,1,1,1,0.0,0.500000
3,b,10050.000000,2,2,2,1.0,1.000000
4,c,11500.000000,1,1,1,0.0,0.666667
5,c,11500.000000,2,1,1,0.0,0.666667
6,c,11666.666667,3,3,2,1.0,1.000000


In [10]:
myquery="""
SELECT 
grade, 
LAG(grade) OVER win      as lag_val_1,
LAG(grade,2) OVER win    as lag_val_2,
LEAD(grade) OVER win     as lead_val_1,
LEAD(grade,2) OVER win   as lead_val_2,
FIRST_VALUE(grade) OVER win as first_val,
LAST_VALUE(grade) OVER win  as last_val
FROM
salaryTable 
WINDOW win as (PARTITION BY grade ORDER BY salary)
""" 
pd.read_sql(myquery, conn)

,grade,lag_val_1,lag_val_2,lead_val_1,lead_val_2,first_val,last_val
0,a,None,None,a,None,a,a
1,a,a,None,None,None,a,a
2,b,None,None,b,None,b,b
3,b,b,None,None,None,b,b
4,c,None,None,c,c,c,c
5,c,c,None,c,None,c,c
6,c,c,c,None,None,c,c


In [11]:
myquery="""
SELECT * FROM
salaryTable 
WHERE salary in
(SELECT 
f1.unique_salary
FROM
(SELECT 
salary as unique_salary,
COUNT(*) as count_unique_salary
FROM salaryTable
GROUP BY salary 
ORDER BY salary DESC) as f1
WHERE f1.count_unique_salary==1 
ORDER BY f1.unique_salary DESC
LIMIT 1)
""" 
pd.read_sql(myquery, conn)

,index,empid,project,grade,salary,variable
0,6,390,p2,b,10100,140


### Example 2

In [12]:
dataDict =  {
    'user_id' : [1,2,3,4],
    'start_date':['2019-01-01','2019-01-15','2019-01-29','2019-02-05'],
    'end_date':['2019-01-31','2019-01-17','2019-02-04','2019-02-10']
}
df3 = pd.DataFrame(dataDict)
df3

,user_id,start_date,end_date
0,1,2019-01-01,2019-01-31
1,2,2019-01-15,2019-01-17
2,3,2019-01-29,2019-02-04
3,4,2019-02-05,2019-02-10


In [13]:
conn = sqlite3.connect(':memory:')
df3.to_sql('subscriptionTable', conn)

4

In [14]:
myquery="""
SELECT 
min(start_date) as min_date, 
max(start_date) as max_date
FROM subscriptionTable
""" 
pd.read_sql(myquery, conn)

,min_date,max_date
0,2019-01-01,2019-02-05


In [15]:
myquery="""
WITH f1 as
(SELECT 
min(start_date) as min_date, 
max(start_date) as max_date
FROM subscriptionTable)
SELECT
f2.user_id,
CASE WHEN f2.end_date >= f1.min_date AND f2.end_date <= f1.max_date THEN 1
ELSE 0
END as overlap
FROM subscriptionTable as f2
JOIN f1
""" 
pd.read_sql(myquery, conn)

,user_id,overlap
0,1,1
1,2,1
2,3,1
3,4,0


### Example 3

In [16]:
dataDict =  {
    'user_id' : [1,2,3,4,1,3],
    'start_date':['2019-01-01','2019-01-15','2019-01-29','2019-02-05','2019-01-10','2019-01-29'],
    'end_date':['2019-01-31','2019-01-17','2019-02-04','2019-02-10', '2019-01-17', '2019-01-27']
}
df4 = pd.DataFrame(dataDict)
df4

,user_id,start_date,end_date
0,1,2019-01-01,2019-01-31
1,2,2019-01-15,2019-01-17
2,3,2019-01-29,2019-02-04
3,4,2019-02-05,2019-02-10
4,1,2019-01-10,2019-01-17
5,3,2019-01-29,2019-01-27


In [17]:
conn = sqlite3.connect(':memory:')
df4.to_sql('subscriptionTable', conn)

6

In [18]:
myquery="""
WITH f1 as
(SELECT 
min(start_date) as min_date,
max(start_date) as max_date
FROM subscriptionTable)
SELECT 
f.user_id,
f.start_date,
CASE WHEN f.end_date>=f1.min_date AND f.end_date<=f1.max_date THEN 1
ELSE 0
END as accept
FROM subscriptionTable as f
JOIN f1
"""
pd.read_sql(myquery, conn)

,user_id,start_date,accept
0,1,2019-01-01,1
1,2,2019-01-15,1
2,3,2019-01-29,1
3,4,2019-02-05,0
4,1,2019-01-10,1
5,3,2019-01-29,1


In [19]:
myquery="""
WITH f1 as
(SELECT 
min(start_date) as min_date,
max(start_date) as max_date
FROM subscriptionTable),
f2 as
(SELECT 
f.user_id,
f.start_date,
CASE WHEN f.end_date>=f1.min_date AND f.end_date<=f1.max_date THEN 1
ELSE 0
END as accept
FROM subscriptionTable as f
JOIN f1)
SELECT 
user_id, 
start_date, 
SUM(accept) as total_accept
FROM f2
WHERE accept IS NOT 0
GROUP BY user_id, start_date
"""
pd.read_sql(myquery, conn)

,user_id,start_date,total_accept
0,1,2019-01-01,1
1,1,2019-01-10,1
2,2,2019-01-15,1
3,3,2019-01-29,2


### Example 4 : Self-Join

In [20]:
dataDict =  {
    'empid' : [1,2,3,4,5],
    'employee':['a','b','c','d','e'],
    'manager':['b','f','b','b','d']
}
df4  = pd.DataFrame(dataDict)
df4

,empid,employee,manager
0,1,a,b
1,2,b,f
2,3,c,b
3,4,d,b
4,5,e,d


In [21]:
conn = sqlite3.connect(':memory:')

df4.to_sql('employeeTable', conn)

5

In [22]:
myquery="""
WITH f1 as
(SELECT 
empid,
employee  
FROM employeeTable),
f2 as
(SELECT 
manager as manager
FROM employeeTable)
SELECT
f2.manager,
f1.empid as empid
FROM f2
LEFT JOIN f1
ON f2.manager = f1.employee 
""" 
pd.read_sql(myquery, conn)

,manager,empid
0,b,2.0
1,f,NaN
2,b,2.0
3,b,2.0
4,d,4.0


In [23]:
conn.close()